## Task1 (Coding)

In this task, we will implement a full ML classifier based on decision trees (in python using Jupyter notebook). We will use the Mushroom Data Set to train and evaluate your classifier. This dataset comes from the UCI ML repository. (Hint: There are missing values in this dataset. At this particular time, you may ignore instances that have missing values and just remove them, or replace missing values with the mean value of the column. Please note that there are other ways of pre-processing data which we have not seen yet.)

You can use libraries e.g., Pandas, NumPy but you may NOT use any prebuilt decision tree packages. 

In [1]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
mushroom = fetch_ucirepo(id=73) 
  
# data (as pandas dataframes) 
X = mushroom.data.features 
y = mushroom.data.targets 
  
# metadata 
print(mushroom.metadata) 
  
# variable information 
print(mushroom.variables) 


{'uci_id': 73, 'name': 'Mushroom', 'repository_url': 'https://archive.ics.uci.edu/dataset/73/mushroom', 'data_url': 'https://archive.ics.uci.edu/static/public/73/data.csv', 'abstract': 'From Audobon Society Field Guide; mushrooms described in terms of physical characteristics; classification: poisonous or edible', 'area': 'Biology', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 8124, 'num_features': 22, 'feature_types': ['Categorical'], 'demographics': [], 'target_col': ['poisonous'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1981, 'last_updated': 'Thu Aug 10 2023', 'dataset_doi': '10.24432/C5959T', 'creators': [], 'intro_paper': None, 'additional_info': {'summary': "This data set includes descriptions of hypothetical samples corresponding to 23 species of gilled mushrooms in the Agaricus and Lepiota Family (pp. 500-525).  Each species is identified as definitely edible, definitely po

Implement the basic decision tree procedure as discussed in the lectures. 
Implement 'DecisionTree' algorithm with a train procedure. 
Implement the information gain criterion as described in our lectures. 

In your report use one or two sentences to discuss the output (the output of the training procedure is the trained decision tree which is a representation of the if-then-else rules). 

You may print out your decision tree (you don't have to, however it might help you discuss the trained trees)  (This may be large - consider the best way to print it). 

In [2]:
import pandas as pd

left = pd.DataFrame(X)
right = pd.DataFrame(y)
# df = pd.concat([left, right])
df = pd.DataFrame.dropna(pd.concat([left, right], axis=1)).reset_index().drop('index',axis=1)
df

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,poisonous
0,x,s,n,t,p,f,c,n,k,e,...,w,w,p,w,o,p,k,s,u,p
1,x,s,y,t,a,f,c,b,k,e,...,w,w,p,w,o,p,n,n,g,e
2,b,s,w,t,l,f,c,b,n,e,...,w,w,p,w,o,p,n,n,m,e
3,x,y,w,t,p,f,c,n,n,e,...,w,w,p,w,o,p,k,s,u,p
4,x,s,g,f,n,f,w,b,k,t,...,w,w,p,w,o,e,n,a,g,e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5639,b,y,n,f,n,f,c,b,w,e,...,n,n,p,w,t,p,w,y,p,e
5640,x,y,n,f,n,f,c,b,w,e,...,n,n,p,w,t,p,w,y,p,e
5641,x,y,g,t,n,f,c,b,w,e,...,w,w,p,w,t,p,w,y,p,e
5642,x,y,c,f,m,f,c,b,y,e,...,c,c,p,w,n,n,w,c,d,p


In [35]:
from ucimlrepo import fetch_ucirepo
import numpy as np
import math

class DecisionTree:
    def __init__(self):
        self.ig = {}
    
    def entropy(self, X, y):
        yes = {}
        total = {}
        entropies = {}
        count = 0
        label = np.array(y)
        for col in X:
            for values in X[col]:
                if label[count] == "p":
                    try:
                        yes[values] += 1
                    except KeyError:
                        yes[values] = 1
                try:
                    total[values] += 1
                except KeyError:
                    total[values] = 1
                count += 1

            for key in yes:
                pyes = yes[key]/total[key]
                pno = (total[key]-yes[key])/total[key]
                if pyes == 0:
                    entropies[key] = 0-(pno)*math.log2(pno)
                elif pno == 0:
                    entropies[key] = -(pyes)*math.log2(pyes)
                else:
                    entropies[key] = -(pyes)*math.log2(pyes)-(pno)*math.log2(pno)

            self.ig[col] = self.information_gain(total, entropies)
            entropies = {}
            yes = {}
            total = {}
            count = 0
        print(self.ig)
        return (max(self.ig.items(), key=lambda k :k[1]))
                
    def poi_entropy(self, y):
        yes = 0
        no = 0
        for col in y:
            for values in y[col]:
                if values == "p":
                    yes += 1
                elif values == "e":
                    no += 1
        pyes = yes/np.array(y).shape[0]   
        pno = no/np.array(y).shape[0]    
        return -(pyes)*math.log2(pyes)-(pno)*math.log2(pno)
        
    def information_gain(self, total, entropy):
        col_total = 8124
        ig = self.poi_entropy(y) 
        for key in entropy:
            try:
                if str.isalpha(key):
                    ig -= total[key]/col_total * entropy[key]
            except:
                col_total -= total[key]
        return ig
            
training_data = DecisionTree()
poison = training_data.poi_entropy(y)
info_gain = training_data.entropy(X,y)
print(info_gain)

{'cap-shape': 0.04879670193537283, 'cap-surface': 0.02859023277377265, 'cap-color': 0.03604928297620379, 'bruises': 0.19237948576121955, 'odor': 0.9060749773839998, 'gill-attachment': 0.014165027250616243, 'gill-spacing': 0.10088318399657023, 'gill-size': 0.23015437514804604, 'gill-color': 0.41697752341613115, 'stalk-shape': 0.00751677256966421, 'stalk-root': 0.38813498914764777, 'stalk-surface-above-ring': 0.2847255992184844, 'stalk-surface-below-ring': 0.2718944733927462, 'stalk-color-above-ring': 0.2538451734622398, 'stalk-color-below-ring': 0.24141556652756646, 'veil-type': 0.0, 'veil-color': 0.02381701612091669, 'ring-number': 0.03845266924309048, 'ring-type': 0.3180215107935375, 'spore-print-color': 0.4807049176849153, 'population': 0.20195801906685235, 'habitat': 0.15683360460509196}
('odor', 0.9060749773839998)


In [14]:
from ucimlrepo import fetch_ucirepo
import numpy as np
import math

class DecisionTree:
    def __init__(self):
        self.ig = {}
        self.splits = {}

    def entropy(self, df):
        yes = {}
        total = {}
        entropies = {}
        for features in df:
            if features != 'poisonous' and features not in self.splits:
                for i in range(len(df)):
                    if df.loc[i,'poisonous'] == 'p':
                        try:
                            yes[df.loc[i,features]] += 1
                        except KeyError:
                            yes[df.loc[i,features]] = 1
                    try:
                        total[df.loc[i,features]] += 1
                    except KeyError:
                        total[df.loc[i,features]] = 1
                        
                        
                for key in yes:
                    pyes = yes[key]/total[key]
                    pno = (total[key]-yes[key])/total[key]
                    if pyes == 0:
                        entropies[key] = -(pno)*math.log2(pno)
                    elif pno == 0:
                        entropies[key] = -(pyes)*math.log2(pyes)
                    else:
                        entropies[key] = -(pyes)*math.log2(pyes)-(pno)*math.log2(pno)
                        
                self.ig[features] = self.information_gain(df, total, entropies)
                entropies = {}
                yes = {}
                total = {}
            elif features != 'poisonous' and features in self.splits:
                self.ig[features] = -100
        return (max(self.ig.items(), key=lambda k :k[1]))
    
    def poi_entropy(self, y):
        yes = 0
        no = 0
        for values in y:
            if values == "p":
                yes += 1
            elif values == "e":
                no += 1
        pyes = yes/np.array(y).shape[0]   
        pno = no/np.array(y).shape[0]    
        if pyes == 0:
            return -(pno)*math.log2(pno)
        elif pno == 0:
            return -(pyes)*math.log2(pyes)
        else:
            return -(pyes)*math.log2(pyes)-(pno)*math.log2(pno)
    
    def information_gain(self, df, total, entropy):
        col_total = sum(total.values())
        poison_ig = self.poi_entropy(df['poisonous']) 
        for key in entropy:
            poison_ig -= total[key]/col_total * entropy[key]
        return poison_ig
    
    def split_data(self, info_gain, df):
        unique_values = {feature: df[feature].unique() for feature in df}
        info_gains = {}
        for feature in unique_values[info_gain[0]]:
            split_data = df[df[info_gain[0]] == feature].reset_index().drop('index',axis=1)
            new_info_gain = self.entropy(split_data)
            if new_info_gain[0] in self.splits:
                continue
            elif new_info_gain[1] == 0.0:
                print(feature) 
                ### print feature leaf
                continue

            info_gains[new_info_gain] = split_data
            print(feature, new_info_gain)

        try:
            # Find the key with the highest value in the dictionary
            max_info_gain = max(info_gains, key=info_gains.get)
            return {max_info_gain: info_gains[max_info_gain]}
        except:
            return
    
training_data = DecisionTree()
info_gain = training_data.entropy(df)
training_data.splits[info_gain[0]] = - 100
split_ig = training_data.split_data(info_gain, df)

# Get the key with the highest information gain
highest_ig = max(split_ig, key=split_ig.get)
training_data.splits[highest_ig[0]] = -100
print()
while split_ig:
    highest_ig = max(split_ig, key=split_ig.get)
    split_ig = training_data.split_data(highest_ig, split_ig[highest_ig])
    print()

    

p
a
l
n ('spore-print-color', 0.17897680716377587)
f
c
m

n
k
r
w ('cap-color', 0.5916727785823275)

c
n
w
y
p
g

